In [12]:
import pandas as pd

from pyjstat import pyjstat

In [6]:
url = r"https://www.nomisweb.co.uk/api/v01/dataset/NM_161_1.jsonstat.json?geography=1245710776...1245710790,1245712478...1245712543,1245710706...1245710716,1245715055,1245710717...1245710734,1245714957,1245713863...1245713902,1245710735...1245710751,1245714958,1245715056,1245710752...1245710775,1245709926...1245709950,1245714987,1245714988,1245709951...1245709978,1245715039,1245709979...1245710067,1245710832...1245710868,1245712005...1245712034,1245712047...1245712067,1245711988...1245712004,1245712035...1245712046,1245712068...1245712085,1245710791...1245710831,1245712159...1245712222,1245709240...1245709350,1245715048,1245715058...1245715063,1245709351...1245709382,1245715006,1245709383...1245709577,1245713352...1245713362,1245715027,1245713363...1245713411,1245715017,1245713412...1245713456,1245715030,1245713457...1245713502,1245709578...1245709655,1245715077...1245715079,1245709679...1245709716,1245709656...1245709678,1245709717...1245709758,1245710900...1245710939,1245714960,1245715037,1245715038,1245710869...1245710899,1245714959,1245710940...1245711009,1245713903...1245713953,1245715016,1245713954...1245713977,1245709759...1245709925,1245714949,1245714989,1245714990,1245715014,1245715015,1245710411...1245710660,1245714998,1245715007,1245715021,1245715022,1245710661...1245710705,1245711010...1245711072,1245714961,1245714963,1245714965,1245714996,1245714997,1245711078...1245711112,1245714980,1245715050,1245715051,1245711073...1245711077,1245712223...1245712237,1245714973,1245712238...1245712284,1245714974,1245712285...1245712294,1245715018,1245712295...1245712306,1245714950,1245712307...1245712316,1245715065,1245715066,1245713503...1245713513,1245714966,1245713514...1245713544,1245714962,1245713545...1245713581,1245714964,1245715057,1245713582...1245713587,1245715010,1245715011,1245713588...1245713627,1245715012,1245715013,1245713628...1245713665,1245713774...1245713779,1245715008,1245715009,1245713780...1245713862,1245713978...1245714006,1245715049,1245714007...1245714019,1245715052,1245714020...1245714033,1245714981,1245714034...1245714074,1245711113...1245711135,1245714160...1245714198,1245711159...1245711192,1245711136...1245711158,1245714270...1245714378,1245714616...1245714638,1245714952,1245714639...1245714680,1245710068...1245710190,1245714953,1245714955,1245715041...1245715047,1245710191...1245710231,1245714951,1245710232...1245710311,1245714956,1245710312...1245710339,1245714954,1245710340...1245710410,1245715040,1245714843...1245714927,1245711814...1245711833,1245711797...1245711813,1245711834...1245711849,1245711458...1245711478,1245711438...1245711457,1245715023,1245715024,1245711479...1245711512,1245715005,1245715071,1245711915...1245711936,1245714971,1245711937...1245711987,1245715019,1245715020,1245712611...1245712711,1245715068,1245712712...1245712784,1245713023...1245713175,1245713666...1245713758,1245715053,1245715054,1245713759...1245713773,1245714379...1245714395,1245714972,1245714396...1245714467,1245708449...1245708476,1245708289,1245708620...1245708645,1245715064,1245715067,1245708646...1245708705,1245714941,1245708822...1245708865,1245708886...1245708919,1245714947,1245708920...1245708952,1245714930,1245714931,1245714944,1245708978...1245709014,1245709066...1245709097,1245714948,1245709121...1245709150,1245714999,1245715000,1245709179...1245709239,1245708290...1245708310,1245714945,1245708311...1245708378,1245714932,1245708379...1245708448,1245714929,1245714934,1245714936,1245708477...1245708519,1245714935,1245708520...1245708557,1245714938,1245708558...1245708592,1245714940,1245708593...1245708619,1245714933,1245715072...1245715076,1245708706...1245708733,1245714942,1245715028,1245708734...1245708794,1245714943,1245708795...1245708821,1245714939,1245708866...1245708885,1245708953...1245708977,1245709015...1245709042,1245714946,1245715069,1245715070,1245709043...1245709065,1245709098...1245709120,1245714982,1245709151...1245709178,1245711551...1245711565,1245711690...1245711722,1245711779...1245711796,1245711513...1245711550,1245711658...1245711689,1245711723...1245711746,1245714967,1245711588...1245711619,1245711747...1245711778,1245711566...1245711587,1245711620...1245711657,1245711850...1245711884,1245714969,1245711885...1245711914,1245714970,1245712544...1245712554,1245715003,1245715004,1245712555...1245712610,1245712860...1245712894,1245714975,1245714984,1245712895...1245712958,1245714968,1245714976,1245714977,1245712959...1245713022,1245713176...1245713206,1245715001,1245715002,1245713207...1245713279,1245714978,1245713280...1245713291,1245715025,1245715026,1245713292...1245713337,1245714979,1245713338...1245713351,1245714075...1245714144,1245715032,1245714145...1245714159,1245714468...1245714493,1245714983,1245714494...1245714587,1245714937,1245714588...1245714603,1245714985,1245714604...1245714615,1245714681...1245714780,1245711193...1245711219,1245711375...1245711395,1245715029,1245715031,1245711220...1245711270,1245715033...1245715036,1245712086...1245712158,1245714928,1245711271...1245711294,1245714991,1245714992,1245711327...1245711358,1245711396...1245711413,1245711295...1245711326,1245711414...1245711437,1245714993...1245714995,1245711359...1245711374,1245714986,1245714781...1245714842,1245712317...1245712477,1245712785...1245712859,1245714199...1245714269,1245715080...1245715134,1245715485,1245715135...1245715171,1245715486,1245715172...1245715188,1245715480,1245715482,1245715189...1245715196,1245715487,1245715197...1245715236,1245715484,1245715237...1245715285,1245715483,1245715286...1245715319,1245715434...1245715479,1245715488,1245715489,1245715320...1245715356,1245715481,1245715357...1245715433&date=latestMINUS1&cause_of_death=9&gender=1,2&age=0...20&measure=1&measures=20100"

In [7]:
dataset = pyjstat.Dataset.read(url)

In [8]:
df = dataset.write('dataframe')

In [9]:
print(df)

       date        geography                                 cause of death  \
0      2018  East Dorset 009  I00-I99 IX Diseases of the circulatory system   
1      2018  East Dorset 009  I00-I99 IX Diseases of the circulatory system   
2      2018  East Dorset 009  I00-I99 IX Diseases of the circulatory system   
3      2018  East Dorset 009  I00-I99 IX Diseases of the circulatory system   
4      2018  East Dorset 009  I00-I99 IX Diseases of the circulatory system   
...     ...              ...                                            ...   
24985  2018      Newport 020  I00-I99 IX Diseases of the circulatory system   
24986  2018      Newport 020  I00-I99 IX Diseases of the circulatory system   
24987  2018      Newport 020  I00-I99 IX Diseases of the circulatory system   
24988  2018      Newport 020  I00-I99 IX Diseases of the circulatory system   
24989  2018      Newport 020  I00-I99 IX Diseases of the circulatory system   

       Gender               Age measure measures  v

In [10]:
df.to_csv("./X_Output/data.csv")

In [5]:
la_geography = [
    "1807745025...1807745028",
    "1807745030...1807745032",
    "1807745034...1807745083",
    "1807745085",
    "1807745282",
    "1807745283",
    "1807745086...1807745155",
    "1807745157...1807745164",
    "1807745166...1807745170",
    "1807745172...1807745177",
    "1807745179...1807745194",
    "1807745196",
    "1807745197",
    "1807745199",
    "1807745201...1807745206",
    "1811939329...1811939332",
    "1811939334...1811939336",
    "1811939338...1811939428",
    "1811939436...1811939442",
    "1811939768",
    "1811939769",
    "1811939443...1811939497",
    "1811939499...1811939501",
    "1811939503",
    "1811939505...1811939507",
    "1811939509...1811939517",
    "1811939519",
    "1811939520",
    "1811939524...1811939570",
    "1811939575...1811939599",
    "1811939601...1811939628",
    "1811939630...1811939634",
    "1811939636...1811939647",
    "1811939649",
    "1811939655...1811939664",
    "1811939667...1811939680",
    "1811939682",
    "1811939683",
    "1811939685",
    "1811939687...1811939692",
]
geography_enc = ",".join(la_geography)

In [7]:
dates = [
    "latest",
    "latestMINUS1",
    "latestMINUS2",
    "latestMINUS3",
    "latestMINUS4",
    "latestMINUS5",
    "latestMINUS6",
]
date_enc = ",".join(dates)

In [8]:
url_base = "https://www.nomisweb.co.uk/api/v01/dataset/NM_161_1.jsonstat.json?"
url_geography_base="geography="
url_date_base="date="
url_params = "&cause_of_death=0,9&gender=0...2&age=0...20&measure=1&measures=20100"

In [9]:
url = (
    url_base
    + url_geography_base
    + geography_enc
    + url_date_base
    + date_enc
    + url_params
)

In [10]:
url

'https://www.nomisweb.co.uk/api/v01/dataset/NM_161_1.jsonstat.json?geography=1807745025...1807745028,1807745030...1807745032,1807745034...1807745083,1807745085,1807745282,1807745283,1807745086...1807745155,1807745157...1807745164,1807745166...1807745170,1807745172...1807745177,1807745179...1807745194,1807745196,1807745197,1807745199,1807745201...1807745206,1811939329...1811939332,1811939334...1811939336,1811939338...1811939428,1811939436...1811939442,1811939768,1811939769,1811939443...1811939497,1811939499...1811939501,1811939503,1811939505...1811939507,1811939509...1811939517,1811939519,1811939520,1811939524...1811939570,1811939575...1811939599,1811939601...1811939628,1811939630...1811939634,1811939636...1811939647,1811939649,1811939655...1811939664,1811939667...1811939680,1811939682,1811939683,1811939685,1811939687...1811939692date=latest,latestMINUS1,latestMINUS2,latestMINUS3,latestMINUS4,latestMINUS5,latestMINUS6&cause_of_death=0,9&gender=0...2&age=0...20&measure=1&measures=20100'

In [13]:
dataset = pyjstat.Dataset.read(url)

In [14]:
df = dataset.write('dataframe')

In [15]:
df.to_csv("./X_Output/la_data.csv")